In [19]:
import pandas as pd
import numpy as np

In [56]:
np.random.seed(8)
df = pd.DataFrame(np.random.rand(5, 4))
df

,0,1,2,3
0,0.873429,0.968541,0.869195,0.530856
1,0.232728,0.011399,0.430469,0.402351
2,0.522675,0.478392,0.555356,0.543386
3,0.760896,0.712375,0.619682,0.426092
4,0.289075,0.973855,0.333774,0.218801


In [57]:
class AgglomerativeClustering:
    def __init__(self, df, n_cluster, linkage):

        # Init each data as a cluster
        df['Cluster'] = pd.DataFrame(np.arange(len(df)), columns=['Cluster'])
        
        # Init important variable
        self.arr = np.array(df)
        self.idxCluster = self.arr.shape[1] - 1
        self.cntCluster = self.arr.shape[0]
        self.n_cluster = n_cluster
        self.linkage = linkage
        self.m = np.zeros((self.cntCluster, self.cntCluster))
        self.initDistanceMatrix()

    def euclidean(self, row1, row2):
        return np.sqrt(np.sum((row1[:-1] - row2[:-1]) ** 2, axis = 0))
    
    def initDistanceMatrix(self):
        for i in range(self.m.shape[0]):
            for j in range(self.m.shape[0]):
                self.m[i, j] = self.euclidean(self.arr[i], self.arr[j])
        
    def printAll(self):
        print(self.arr)
        print(self.m)
    
    def findIdxCluster(self, idx):
        return np.where(self.arr[:, self.idxCluster] == idx)[0][0]
        
    def makeCluster(self, idx1, idx2):
        
        # Change cluster
        idxChange = np.where(self.arr[:, self.idxCluster] == idx2)
        
        for i in idxChange:
            self.arr[i, self.idxCluster] = self.arr[idx1, self.idxCluster]
        
        # Update cluster num
        idxCluster = np.where(self.arr[:, self.idxCluster] > idx2)
        
        for j in idxCluster:
            self.arr[j, self.idxCluster] -= 1
        
        # Update count cluster
        self.cntCluster -= 1
    
    def isMoreThanOne(self, num, arr):
        cnt = 0
        for i in arr:
            if i == num:
                cnt += 1
        return cnt > 1
    
    def checkLinkage(self):
        linkage = ['single', 'complete', 'average', 'average-group']
        return self.linkage in linkage
            
    def fit_predict(self):
        if self.checkLinkage():
            if self.n_cluster > 1:
                while (self.cntCluster > self.n_cluster):

                    # Create new cluster
                    minVal = np.min(self.m[self.m > 0])
                    idx = np.where(np.isclose(self.m, minVal))
                    self.makeCluster(idx[0][0], idx[1][0])

                    # Create new distance matrix
                    self.m = np.zeros((self.cntCluster, self.cntCluster))

                    # i, j are cluster label
                    for i in range(self.m.shape[0]):
                        for j in range(self.m.shape[0]):
                            if (i == j):
                                self.m[i, j] = 0
                            else:
                                # ONE TO ONE
                                if not self.isMoreThanOne(i, self.arr[:, self.idxCluster]) and not self.isMoreThanOne(j, self.arr[:, self.idxCluster]):
                                    self.m[i, j] = self.euclidean(self.arr[self.findIdxCluster(i)], self.arr[self.findIdxCluster(j)])
                                else:
                                    both = False
                                    # MANY TO MANY
                                    if self.isMoreThanOne(i, self.arr[:, self.idxCluster]) and self.isMoreThanOne(j, self.arr[:, self.idxCluster]):
                                        row, col = np.where(self.arr == i)
                                        idxI = row[np.where(col == self.idxCluster)]
                                        row, col = np.where(self.arr == j)
                                        idxJ = row[np.where(col == self.idxCluster)]
                                        temp = []
                                        for k in idxI:
                                            for l in idxJ:
                                                temp.append(self.euclidean(self.arr[k], self.arr[l]))
                                        both = True                                               
                                    # ONE TO MANY/MANY TO ONE
                                    elif self.isMoreThanOne(i, self.arr[:, self.idxCluster]):
                                        row, col = np.where(self.arr == i)
                                        idx = row[np.where(col == self.idxCluster)]
                                        temp = []
                                        for k in idx:
                                            temp.append(self.euclidean(self.arr[k], self.arr[self.findIdxCluster(j)]))
                                    elif self.isMoreThanOne(j, self.arr[:, self.idxCluster]):
                                        row, col = np.where(self.arr == j)
                                        idx = row[np.where(col == self.idxCluster)]
                                        temp = []
                                        for k in idx:
                                            temp.append(self.euclidean(self.arr[k], self.arr[self.findIdxCluster(i)]))
                                    if self.linkage == 'single':
                                        self.m[i, j] = np.min(temp)
                                    elif self.linkage == 'complete':
                                        self.m[i, j] = np.max(temp)
                                    elif self.linkage == 'average':
                                        self.m[i, j] = np.mean(temp)
                                    elif self.linkage == 'average-group':
                                        if both:
                                            for k in range(0, len(idxI)):
                                                for l in range(k + 1, len(idxI)):
                                                    temp.append(self.euclidean(self.arr[idxI[k]], self.arr[idxI[l]]))
                                            for k in range(0, len(idxJ)):
                                                for l in range(k + 1, len(idxJ)):
                                                    temp.append(self.euclidean(self.arr[idxJ[k]], self.arr[idxJ[l]]))
                                        else:
                                            for k in range(0, len(idx)):
                                                for l in range(k + 1, len(idx)):
                                                    temp.append(self.euclidean(self.arr[idx[k]], self.arr[idx[l]]))
                                        self.m[i, j] = np.mean(temp)
            else:
                self.arr[:, self.idxCluster] = 0
        else:
            print('Wrong Argument!')
        
#         self.printAll()
        
        return self.arr[:, self.idxCluster].astype('int32')
        
        

print(AgglomerativeClustering(df, 2, 'average').fit_predict())


[0 1 0 0 0]


In [71]:
import numpy as np
import pandas as pd
import sklearn.cluster as sklearn_cluster
from sklearn import datasets
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = iris.data
y = iris.target
n_clusters = len(iris.target_names)

print("\n===========================\n")

print("Agglomerative Clustering (Single) from Scratch")
y_predict = AgglomerativeClustering(pd.DataFrame(X), n_clusters, 'single').fit_predict()
print(y_predict)

print ('Confusion Matrix :', confusion_matrix(y, y_predict))
print ('Accuracy Score :', accuracy_score(y, y_predict))

print("\n===========================\n")

print("Agglomerative Clustering (Single) SKLearn")
y_predict = sklearn_cluster.AgglomerativeClustering(linkage='single').fit_predict(X)
print(y_predict)

print ('Confusion Matrix :', confusion_matrix(y, y_predict))
print ('Accuracy Score :', accuracy_score(y, y_predict))

print("\n===========================\n")

print("Agglomerative Clustering (Complete) from Scratch")
y_predict = AgglomerativeClustering(pd.DataFrame(X), n_clusters, 'complete').fit_predict()
print(y_predict)

print ('Confusion Matrix :', confusion_matrix(y, y_predict))
print ('Accuracy Score :', accuracy_score(y, y_predict))

print("\n===========================\n")

print("Agglomerative Clustering (Complete) SKLearn")
y_predict = sklearn_cluster.AgglomerativeClustering(linkage='complete').fit_predict(X)
print(y_predict)

print ('Confusion Matrix :', confusion_matrix(y, y_predict))
print ('Accuracy Score :', accuracy_score(y, y_predict))

print("\n===========================\n")

print("Agglomerative Clustering (Average) from Scratch")
y_predict = AgglomerativeClustering(pd.DataFrame(X), n_clusters, 'average').fit_predict()
print(y_predict)

print ('Confusion Matrix :', confusion_matrix(y, y_predict))
print ('Accuracy Score :', accuracy_score(y, y_predict))

print("\n===========================\n")

print("Agglomerative Clustering (Average) SKLearn")
y_predict = sklearn_cluster.AgglomerativeClustering(linkage='average').fit_predict(X)
print(y_predict)

print ('Confusion Matrix :', confusion_matrix(y, y_predict))
print ('Accuracy Score :', accuracy_score(y, y_predict))

print("\n===========================\n")

print("Agglomerative Clustering (Average-Group) from Scratch")
y_predict = AgglomerativeClustering(pd.DataFrame(X), n_clusters, 'average-group').fit_predict()
print(y_predict)

print ('Confusion Matrix :', confusion_matrix(y, y_predict))
print ('Accuracy Score :', accuracy_score(y, y_predict))



Agglomerative Clustering (Single) from Scratch
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0
 0 0]
Confusion Matrix : [[50  0  0]
 [50  0  0]
 [48  1  1]]
Accuracy Score : 0.34


Agglomerative Clustering (Single) SKLearn
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
Confusion Matrix : [[ 0 50  0]
 [50  0  0]
 [50  0  0]]
Accuracy Score : 0.0


Agglomerative Clustering (Complete) from Scratch
[0 1 2 2 0 0 2 0 2 1 2 2 1 2 0 0 0 0 0 0 2 0 1 0 2 1 0 1 2 2 1 0 2 2 1 2 2
 0 2 0 0 1 2 0